In [1]:
import sys
sys.path.insert(1, '..\\..\\')

import os
import data_loader
from numpy import trapz
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Tensorflow Version

In [2]:
print(tf.__version__)

2.3.1


### Global Variables

In [3]:
malware_data_dir = '../../data/'
saved_model_path = 'saved_model/'
opcode_to_int_path = "opcodeToInt.txt"
num_unique_opcodes = 50
max_opcode_sequence_length = 4000
embed_vector_length = 128
num_lstm_unit = 16
dropout_amt = 0.3
batch_size = 64
num_epochs = 100
test_size= 0.15       # reserve for testing
num_families_to_use = 10

shutdown = False

### Load the Data

In [4]:
raw_train_data = data_loader.getTrainData(malware_data_dir, 
                                          num_families_to_use, 
                                          num_unique_opcodes, 
                                          max_opcode_sequence_length, 
                                          opcode_to_int_path)

Getting list of paths to training data
{'winwebsec': 13045263, 'vundo': 6518146, 'zbot': 6348792, 'hotbar': 5904000, 'onlinegames': 5049791, 'renos': 5003745, 'obfuscator': 4798215, 'bho': 4629982, 'zeroaccess': 4476000, 'alureon': 3870723}
Finding out how many opcodes to use per family...
3870723
Generating opcode to int mapping...
File saved, done.
Loading training data for hotbar
Loading training data for renos
Loading training data for vundo
Loading training data for winwebsec
Loading training data for zbot
Loading training data for alureon
Loading training data for bho
Loading training data for obfuscator
Loading training data for onlinegames
Loading training data for zeroaccess
All training data loaded


### Data preprocessing

In [5]:
family_names = list(raw_train_data.keys())
print(family_names)

# Split opcode family data in individual lists
train_data = list()
for family, data in raw_train_data.items():
    train_data.append(data)
    
# Pad training data to ensure uniformity
padded_train_data = list()
for family_opcodes in train_data:
    padded_sequence = pad_sequences(family_opcodes, 
                                    maxlen=max_opcode_sequence_length)
    padded_train_data.append(padded_sequence)
    
# Concatenate all training data into 1 long list instead of multiple lists
train_data_raw = np.concatenate(padded_train_data)

print(len(train_data))

['hotbar', 'renos', 'vundo', 'winwebsec', 'zbot', 'alureon', 'bho', 'obfuscator', 'onlinegames', 'zeroaccess']
10


### Make the labels

In [6]:
train_labels = []

for count, data in enumerate(padded_train_data):
    labels_list = np.full(shape=(len(data)), fill_value=count)
    train_labels.append(labels_list)

train_labels_raw = np.concatenate(train_labels)

### plit into training and testing sets

In [7]:
def split_data(train_data_raw, train_labels_raw):
    # Split into training and testing data
    train_data, test_data, train_labels, test_labels = train_test_split(train_data_raw, train_labels_raw, test_size=test_size)

    # Make divisible by batch size
    num_data_train = int(len(train_data)/batch_size) * batch_size
    num_data_test = int(len(test_data)/batch_size) * batch_size

    train_data = train_data[:num_data_train]
    train_labels = train_labels[:num_data_train]
    test_data = test_data[:num_data_test]
    test_labels = test_labels[:num_data_test]
    
    print("train_data shape: {}".format(train_data.shape))
    print("test_data shape: {}".format(test_data.shape))
    print("train_labels shape: {}".format(train_labels.shape))
    print("test_labels shape: {}".format(test_labels.shape))
    
    return train_data, test_data, train_labels, test_labels

### Make the Model

In [8]:
def create_model():
    model = Sequential()
    model.add(Input(batch_shape=(batch_size, max_opcode_sequence_length), name="input"))
    model.add(Embedding(input_dim=num_unique_opcodes+1,
                                  output_dim=embed_vector_length,
                                  input_length=max_opcode_sequence_length, name="embedding"))
    # LSTM 1
    model.add(Dropout(dropout_amt, name="dropout_1"))
    model.add(LSTM(num_lstm_unit, 
                   input_shape=(None, max_opcode_sequence_length),
                   return_sequences=True,
                   name="lstm1"))
    # LSTM 2
    model.add(Dropout(dropout_amt, name="dropout_2"))
    model.add(LSTM(num_lstm_unit*3, 
                   return_sequences=True,
                   name="lstm2"))
    # LSTM 3
    model.add(Dropout(dropout_amt*2, name="dropout_3"))
    model.add(LSTM(num_lstm_unit, 
                                   name="lstm3"))
    model.add(Dense(num_families_to_use, activation='softmax', name="dense"))
    optimizer = Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #model.summary()
    
    return model

In [9]:
results = []

for i in range(5):
    # get train and test data
    train_data, test_data, train_labels, test_labels = split_data(train_data_raw, train_labels_raw)
    
    # train model
    model_train = create_model()
    early_stopping = EarlyStopping(monitor='loss', verbose=1, patience=2)
    history = model_train.fit(x=train_data,
                          y=train_labels,
                          batch_size=batch_size,
                          callbacks = [early_stopping],
                          epochs=num_epochs,)
    
    # evaluate
    model_evaluate = create_model()
    model_evaluate.set_weights(model_train.get_weights())

    scores = model_evaluate.evaluate(test_data, test_labels, verbose=0, callbacks = [early_stopping])
    accuracy = scores[1]*100
    print(accuracy)
    results.append(accuracy)

train_data shape: (8896, 4000)
test_data shape: (1536, 4000)
train_labels shape: (8896,)
test_labels shape: (1536,)
Epoch 1/100
139/139 [==============================] - 51s 369ms/step - loss: 2.0519 - accuracy: 0.2328
Epoch 2/100
139/139 [==============================] - 52s 371ms/step - loss: 1.8212 - accuracy: 0.3504
Epoch 3/100
139/139 [==============================] - 51s 366ms/step - loss: 1.6470 - accuracy: 0.4288
Epoch 4/100
139/139 [==============================] - 51s 365ms/step - loss: 1.5060 - accuracy: 0.4713
Epoch 5/100
139/139 [==============================] - 51s 366ms/step - loss: 1.4482 - accuracy: 0.4858
Epoch 6/100
136/139 [============================>.] - ETA: 1s - loss: 1.3887 - accuracy: 0.5098

KeyboardInterrupt: 

In [ ]:
# with open("accuracyResults.txt", 'w') as f:
#     for x in results:
#         f.write(str(x) + "\n")

# os.system('shutdown -s')

# model_train.save_weights(saved_model_path) 

### Use History to plot and accuracy throughout training

In [ ]:
history.history

### Evaluate Model

In [ ]:
model_evaluate = create_model()
model_evaluate.set_weights(model_train.get_weights())

scores = model_evaluate.evaluate(test_data, test_labels, verbose=0, callbacks = [early_stopping])
accuracy = scores[1]*100
print("Accuracy: %0.2f%%" % (scores[1]*100))

### Load Model From Save and Evaluate

In [ ]:
model = create_model()
model.load_weights(saved_model_path)



In [ ]:
scores = model.evaluate(test_set, test_labels, verbose=0)
accuracy = scores[1]*100
print("Accuracy: %0.2f%%" % (scores[1]*100))

## Make ROC Curve

### Make scatter plot

In [ ]:
# separate winwebsec and zbot test data
winwebsec_test_data = []
zbot_test_data = []

for i in range(len(test_labels)):
    if test_labels[i] == 0:
        winwebsec_test_data.append(test_set[i])
    else:
        zbot_test_data.append(test_set[i])
        
winwebsec_test_data = np.asarray(winwebsec_test_data[:192])
zbot_test_data = np.asarray(zbot_test_data[:128])

In [ ]:
print(winwebsec_test_data.shape)
print(zbot_test_data.shape)


winwebsecY = model.predict(winwebsec_test_data)
winwebsecX = [i+1 for i in range(len(winwebsec_test_data))]

zbotY = model.predict(zbot_test_data)
zbotX = [i+1 for i in range(len(zbot_test_data))]

In [ ]:
plt.figure(100)
f = plt.scatter(winwebsecX, winwebsecY, marker='o',
                c='darkblue', s=30, label="winwebsec")
plt.scatter(zbotX, zbotY, marker='o', c='red', s=30, label="zbot")
plt.title("Winwebsec vs. Zbot LSTM Prediction Scatter Plot",
          fontsize=18, wrap=True)
f.axes.get_xaxis().set_visible(False)
plt.ylabel("Prediction", fontsize=15)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=12)

### Make ROC Curve

In [ ]:
def sortByFirstItem(item):
    return item[0]

In [ ]:
winwebsecROC = [(data, "winwebsec") for data in winwebsecY]
zbotROC = [(data, "zbot") for data in zbotY]

zbotROC.sort(key=sortByFirstItem)
winwebsecROC.sort(key=sortByFirstItem)

dataROC = zbotROC + winwebsecROC
dataROC.sort(key=sortByFirstItem, reverse=True)

In [ ]:
def calculate_TPR_FPR(thresholdLine, dataROC):
    TP = 0
    FN = 0
    TN = 0
    FP = 0

    for data in dataROC:
        yVal = data[0]
        family = data[1]

        if family == "winwebsec":
            if yVal < thresholdLine:
                TP += 1
            else:
                FN += 1
        elif family == "zbot":
            if yVal > thresholdLine:
                TN += 1
            else:
                FP += 1
    TPR = TP/(TP+FN)
    FPR = 1 - (TN/(TN+FP))

    return TPR, FPR

In [ ]:
def calculateAUC(rocData):
    sum = 0

    # initialization
    prevX = -1
    prevY = -1

    for points in rocData:
        curX = points[0]
        curY = points[1]

        # Skip for first point
        if prevX != -1 and prevY != -1:
            # check if rectangle
            if prevY == curY:
                sum += abs(curX - prevX) * prevY
            # check if trapezoid
            else:
                sum += (curY + prevY) * abs(curX - prevX) * 0.5

        prevX = curX
        prevY = curY

    return sum

In [ ]:
rocX = list()  # used to plot
rocY = list()  # used to plot
rocData = list()    # used to calculate AUC

for entry in dataROC:
    thresholdLine = entry[0]
    TPR, FPR = calculate_TPR_FPR(thresholdLine, dataROC)

    rocX.append(FPR)
    rocY.append(TPR)
    rocData.append([FPR, TPR])

rocData.sort(key=lambda item: (item[0], item[1]), reverse=True)

AUC = round(calculateAUC(rocData), 3)

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

plt.figure(200)
plt.plot(rocX, rocY, marker=".", markersize=8)
plt.title("Winwebsec vs. Zbot LSTM Log Probability ROC", fontsize=18)
plt.xlabel("FPR", fontsize=15)
plt.ylabel("TPR", fontsize=15)
plt.grid()
plt.text(x=0.75, y=0, s="AUC: {0}".format(AUC), fontsize=14, bbox=props)

# show plots
plt.show()

In [ ]:
if shutdown:
    os.system('shutdown -s')